In [3]:
import json
import pandas as pd
import numpy as np

In [2]:
x = {'taxonid':[],'code':[], 'suitability':[], 'season':[], 'majorimportance':[]}
with open('data/sp_habitats.json', 'r') as f:
    data = json.load(f)
for i in data:
    for r in i['result']:
        x['taxonid'].append(i['id'])
        x['code'].append(r['code'])
        x['suitability'].append(r['suitability'])
        x['season'].append(r['season'])
        x['majorimportance'].append(r['majorimportance'])
            
habitat = pd.DataFrame(x)

In [8]:
x = {'taxonid':[],'country':[], 'presence':[], 'origin':[], 'distribution':[]}
with open('data/sp_ctry_occur.json', 'r') as f:
    data = json.load(f)

for i in data:
    for r in i['result']:
        x['taxonid'].append(i['name'])
        x['country'].append(r['code']) #only keep country code to avoid redundancy
        x['presence'].append(r['presence'])
        x['origin'].append(r['origin'])
        x['distribution'].append(r['distribution_code'])
country_occurence = pd.DataFrame(x)
country_occurence['origin'].replace('Origin Uncertain', 'Uncertain', inplace=True)

In [9]:
country_occurence.head()

,taxonid,country,presence,origin,distribution
0,166,GA,Extant,Native,Native
1,219,AF,Extinct Post-1500,Native,Regionally Extinct
2,219,AO,Extant,Native,Native
3,219,BF,Extant,Native,Native
4,219,BI,Extinct Post-1500,Native,Regionally Extinct


In [4]:
x = {'taxonid':[]}
for year in range(2000, 2021):
    x[f'y{year}'] = []
    
with open('data/sp_hist_cats.json', 'r') as f:
    data = json.load(f)

for i in data:
    change_yrs = {}
    ind = 0
    start = int(i['result'][ind]['year'])
    while start >= 2000:
        change_yrs[start] = i['result'][ind]['code']
        try:   #check for next entry
            start = int(i['result'][ind+1]['year']) 
            ind += 1
        except:
            break
    x['taxonid'].append(i['name'])
    curr_cat = i['result'][ind]['code']
    
    for year in range(2000, 2021):
        if year < start: # nan for years before a species was assessed
            x[f'y{year}'].append(None)
        else:
            if year in change_yrs:
                curr_cat = change_yrs[year]
            x[f'y{year}'].append(curr_cat)

historical = pd.DataFrame(x)
to_replace = {'E':'EN', 
              'V':'VU', 
              'K':'DD', 
              'I':'EN',
              'T':'VU',
              'NR':None, 
              'LR/nt':'NT', 
              'nt':'LC', 
              'LR/lc':'LC', 
              'LR/cd':'NT'}
historical.replace(to_replace, inplace=True)

In [11]:
x = {'taxonid': [], 'code': [], 'timing': [], 'scope':[], 'severity':[], 'score':[], 'invasive':[]}
with open('data/sp_threats.json') as f:
    data = json.load(f)

for i in data:
    result = i['result']
    if result == []:
        pass
    else:
        for r in result:
            x['taxonid'].append(i['id'])
            x['code'].append(r['code'])
            x['timing'].append(r['timing'])
            x['scope'].append(r['scope'])
            x['severity'].append(r['severity'])
            x['score'].append(r['score'])
            x['invasive'].append(r['invasive'])

threat = pd.DataFrame(x)
threat['score'].replace(['Unknown', '(Not specified)'], None, inplace=True)
threat['score'].replace('Past Impact', 'P', inplace=True)
threat['score'] = [s[-1] if s else None for s in threat['score']]

for i in threat['taxonid'].unique():
    for code in ['8.1.2', '8.2.2', '8.4.2', '8.5.2']:
        if len(threat[(threat['taxonid']==i)&(threat['code']==code)])>1:
            t = threat[(threat['taxonid']==i)&(threat['code']==code)]
            species = [i for i in t['invasive'] if i]
            inds = list(t.index)
            threat.at[inds[0], 'invasive'] = ", ".join(species)
            threat.drop(inds[1:], inplace=True)

In [12]:
x = {}
with open('data/sp_indiv.json', encoding = 'utf-8') as f:
    data = json.load(f)
cols = data[0].keys()
for c in cols:
    x[c] = []

for i in data:
    for c in cols:
        x[c].append(i[c])
        
assessment = pd.DataFrame(x)
assessment.rename({'order':'order_name', 'class':'class_name'}, axis=1, inplace=True)
assessment = assessment.where(pd.notnull(assessment), None)

In [13]:
x = {'scientific':[], 'common':[], 'primary':[], 'language':[]}
with open('data/sp_common_names.json', encoding='utf-8') as f:
    data = json.load(f)

for i in data:
    result = i['result']
    if result != []:
        for r in result:
            x['scientific'].append(i['name'])
            x['common'].append(r['taxonname'])
            x['primary'].append(r['primary'])
            x['language'].append(r['language'])
common_names = pd.DataFrame(x)

In [12]:
with open('data/allCDspecies.json') as f:
    data = json.load(f)
    
x = {'taxonid':[], 'scientific':[]}
for i in data['result']:
    x['taxonid'].append(i['taxonid'])
    x['scientific'].append(i['scientific_name'])

CDspecies = pd.DataFrame(x)

with open('data/allCFspecies.json') as f:
    data = json.load(f)
    
x = {'taxonid':[], 'scientific':[]}
for i in data['result']:
    x['taxonid'].append(i['taxonid'])
    x['scientific'].append(i['scientific_name'])

CFspecies = pd.DataFrame(x)

with open('data/allCGspecies.json') as f:
    data = json.load(f)
    
x = {'taxonid':[], 'scientific':[]}
for i in data['result']:
    x['taxonid'].append(i['taxonid'])
    x['scientific'].append(i['scientific_name'])

CGspecies = pd.DataFrame(x)

with open('data/allCMspecies.json') as f:
    data = json.load(f)
    
x = {'taxonid':[], 'scientific':[]}
for i in data['result']:
    x['taxonid'].append(i['taxonid'])
    x['scientific'].append(i['scientific_name'])
CMspecies = pd.DataFrame(x)

with open('data/allGAspecies.json') as f:
    data = json.load(f)
    
x = {'taxonid':[], 'scientific':[]}
for i in data['result']:
    x['taxonid'].append(i['taxonid'])
    x['scientific'].append(i['scientific_name'])
    
GAspecies = pd.DataFrame(x)

with open('data/allGQspecies.json') as f:
    data = json.load(f)
    
x = {'taxonid':[], 'scientific':[]}
for i in data['result']:
    x['taxonid'].append(i['taxonid'])
    x['scientific'].append(i['scientific_name'])

GQspecies = pd.DataFrame(x)

In [6]:
import psycopg2 as pg
from psycopg2 import extras

In [9]:
curs.close()
conn.close()

In [12]:
q = '''
SELECT count(*)
FROM assessment
'''
curs = conn.cursor()
curs.execute(q)
curs.fetchall()

[(8749,)]

In [18]:
#number of species endemic to congo basin
q = '''
SELECT COUNT(*)
FROM (
SELECT DISTINCT(co.taxonid)
FROM country_occurrence as co
WHERE co.country IN ('GQ') 
AND co.presence != 'Extinct Post-1500'
GROUP BY co.taxonid
EXCEPT
SELECT DISTINCT(co.taxonid)
FROM country_occurrence as co
WHERE co.country NOT IN ('CD', 'GA', 'CF', 'CM', 'CG', 'GQ')
AND co.presence != 'Extinct Post-1500'
GROUP BY co.taxonid) as extant
'''
curs = conn.cursor()
curs.execute(q)
curs.fetchall()

[(240,)]

In [5]:
q = '''SELECT taxonid, category
FROM assessment
'''
curs = conn.cursor()
curs.execute(q)
cats = curs.fetchall()
cat_dict = {}
for taxonid, cat in cats:
    cat_dict[taxonid] = cat

In [23]:
# query species only found in congo basin countries
#SELECT co.taxonid , string_agg(co.country, ', ')
#FROM country_occurrence as co
#WHERE co.taxonid IN 
cb_countries = ['CD', 'GA', 'CF', 'CM', 'CG', 'GQ']
by_country = {}
cat_counts = {}

for c in cb_countries:
    q = f'''SELECT DISTINCT(co.taxonid)
    FROM country_occurrence as co
    WHERE co.country = %s 
    AND co.presence != 'Extinct Post-1500'
    GROUP BY co.taxonid
    EXCEPT
    SELECT DISTINCT(co.taxonid)
    FROM country_occurrence as co
    WHERE co.country NOT IN ('CD', 'GA', 'CF', 'CM', 'CG', 'GQ')
    AND co.presence != 'Extinct Post-1500'
    '''
    curs = conn.cursor()
    curs.execute(q, (c,))
    by_country[c]=[i[0] for i in curs.fetchall()]
    cat_counts[c]={}
for k,c in by_country.items():
    for species in c:
        species_cat = cat_dict[species]
        try:
            cat_counts[k][species_cat]+=1
        except:
            cat_counts[k][species_cat]=1
for k,v in cat_counts.items():
    print(f'{k}: {v}')

CD: {'EN': 63, 'LC': 526, 'DD': 265, 'VU': 103, 'CR': 33, 'LR/cd': 2, 'NT': 28}
GA: {'VU': 109, 'LC': 294, 'DD': 65, 'EN': 63, 'CR': 12, 'NT': 24, 'LR/nt': 8}
CF: {'DD': 36, 'LC': 184, 'VU': 8, 'NT': 5, 'EN': 1}
CM: {'LC': 299, 'VU': 189, 'CR': 144, 'DD': 95, 'LR/nt': 16, 'EN': 210, 'NT': 32}
CG: {'DD': 52, 'LC': 326, 'CR': 1, 'VU': 40, 'NT': 10, 'EN': 16}
GQ: {'VU': 43, 'LC': 117, 'EN': 38, 'DD': 15, 'NT': 14, 'LR/nt': 8, 'CR': 5}


In [16]:
q = '''
SELECT assessment.class_name, history.y2012, COUNT(*) 
FROM assessment FULL OUTER JOIN history on (assessment.taxonid=history.taxonid), CDspecies
WHERE assessment.taxonid=CDspecies.taxonid
GROUP BY assessment.class_name, history.y2012
ORDER BY assessment.class_name
'''
curs = conn.cursor()
curs.execute(q)
curs.fetchall()

[('ACTINOPTERYGII', 'DD', 170),
 ('ACTINOPTERYGII', 'LC', 769),
 ('ACTINOPTERYGII', None, 601),
 ('ACTINOPTERYGII', 'CR', 1),
 ('ACTINOPTERYGII', 'EN', 16),
 ('ACTINOPTERYGII', 'NT', 11),
 ('ACTINOPTERYGII', 'VU', 54),
 ('AGARICOMYCETES', None, 3),
 ('AMPHIBIA', 'LC', 138),
 ('AMPHIBIA', 'DD', 49),
 ('AMPHIBIA', 'EN', 3),
 ('AMPHIBIA', 'NT', 2),
 ('AMPHIBIA', 'VU', 8),
 ('AMPHIBIA', None, 24),
 ('AMPHIBIA', 'CR', 1),
 ('ARACHNIDA', None, 6),
 ('AVES', 'VU', 21),
 ('AVES', 'LC', 915),
 ('AVES', 'DD', 8),
 ('AVES', 'EN', 15),
 ('AVES', None, 164),
 ('AVES', 'NT', 29),
 ('BIVALVIA', 'DD', 7),
 ('BIVALVIA', 'EN', 3),
 ('BIVALVIA', 'LC', 34),
 ('BIVALVIA', 'VU', 5),
 ('BIVALVIA', None, 6),
 ('CEPHALOPODA', None, 22),
 ('CEPHALOPODA', 'DD', 6),
 ('CEPHALOPODA', 'LC', 2),
 ('CHONDRICHTHYES', 'DD', 14),
 ('CHONDRICHTHYES', 'CR', 2),
 ('CHONDRICHTHYES', 'NT', 2),
 ('CHONDRICHTHYES', 'LC', 4),
 ('CHONDRICHTHYES', 'EN', 3),
 ('CHONDRICHTHYES', 'VU', 8),
 ('CHONDRICHTHYES', None, 5),
 ('CYCADOPSID

In [3]:
curs = conn.cursor()
q = '''
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public'
'''
curs.execute(q)
tables = curs.fetchall()
tables = [i[0] for i in tables]
tables

['cdspecies',
 'cfspecies',
 'cgspecies',
 'cmspecies',
 'gaspecies',
 'gqspecies',
 'common_name',
 'assessment',
 'country_occurrence',
 'habitat',
 'threat',
 'history']

In [11]:
curs = conn.cursor()
drop_table = "DROP TABLE history"
curs.execute(drop_table)
conn.commit()
curs.close()

In [18]:
create_assessments = '''
CREATE TABLE assessment (
taxonid INT PRIMARY KEY,
scientific_name VARCHAR(60) UNIQUE,
kingdom VARCHAR(8),
phylum VARCHAR(13),
class_name VARCHAR(14),
order_name VARCHAR(20),
family VARCHAR(20),
genus VARCHAR(25),
main_common_name TEXT,
authority TEXT,
published_year SMALLINT,
assessment_date DATE,
category VARCHAR(5),
criteria TEXT,
population_trend VARCHAR(10),
marine_system BOOLEAN,
freshwater_system BOOLEAN,
terrestrial_system BOOLEAN,
assessor TEXT,
reviewer TEXT,
aoo_km2 TEXT,
eoo_km2 TEXT,
elevation_upper REAL,
elevation_lower REAL,
depth_upper REAL,
depth_lower REAL,
errata_flag BOOLEAN,
errata_reason TEXT,
amended_flag BOOLEAN,
amended_reason TEXT
)
'''
curs = conn.cursor()
curs.execute(create_assessments)
conn.commit()
curs.close()

In [17]:
countries = ['CD', 'CF', 'CG', 'CM', 'GA', 'GQ']
curs = conn.cursor()
for i in countries:
    create_stmt = f'''
    CREATE TABLE {i}species (
    taxonid INT PRIMARY KEY,
    scientific VARCHAR(60)
    )
    '''
    curs.execute(create_stmt)

conn.commit()
curs.close()

In [18]:
curs = conn.cursor()

insert_stmt = 'INSERT INTO CDspecies VALUES %s'
extras.execute_values(curs, insert_stmt, [tuple(i) for i in CDspecies.values])
conn.commit()

insert_stmt = 'INSERT INTO CFspecies VALUES %s'
extras.execute_values(curs, insert_stmt, [tuple(i) for i in CFspecies.values])
conn.commit()

insert_stmt = 'INSERT INTO CGspecies VALUES %s'
extras.execute_values(curs, insert_stmt, [tuple(i) for i in CGspecies.values])
conn.commit()

insert_stmt = 'INSERT INTO CMspecies VALUES %s'
extras.execute_values(curs, insert_stmt, [tuple(i) for i in CMspecies.values])
conn.commit()

insert_stmt = 'INSERT INTO GAspecies VALUES %s'
extras.execute_values(curs, insert_stmt, [tuple(i) for i in GAspecies.values])
conn.commit()

insert_stmt = 'INSERT INTO GQspecies VALUES %s'
extras.execute_values(curs, insert_stmt, [tuple(i) for i in GQspecies.values])
conn.commit()

In [19]:
#populate the assessments table
insert_stmt = f'INSERT INTO assessment ({",".join(list(assessment))}) VALUES %s'
curs = conn.cursor()
extras.execute_values(curs, insert_stmt, [tuple(i) for i in assessment.values])
conn.commit()  

In [20]:
create_country_occ = """
CREATE TABLE country_occurrence (
taxonid INT REFERENCES assessment(taxonid),
country VARCHAR(2),
presence VARCHAR(17),
origin VARCHAR(12),
distribution VARCHAR(26),
PRIMARY KEY (taxonid, country)
)
"""
curs = conn.cursor()
curs.execute(create_country_occ)
conn.commit()
curs.close()

In [21]:
insert_stmt = f'INSERT INTO country_occurrence ({",".join(list(country_occurence))}) VALUES %s'
curs = conn.cursor()
extras.execute_values(curs, insert_stmt, [tuple(i) for i in country_occurence.values])
conn.commit()

In [23]:
create_habitat = """
CREATE TABLE habitat (
taxonid INT REFERENCES assessment(taxonid),
code VARCHAR(6),
suitability VARCHAR(8),
season VARCHAR(27),
majorimportance VARCHAR(3),
PRIMARY KEY (taxonid, code)
)
"""
curs = conn.cursor()
curs.execute(create_habitat)
conn.commit()
curs.close()

In [24]:
insert_stmt = f'''INSERT INTO habitat ({",".join(list(habitat))}) VALUES %s
ON CONFLICT DO NOTHING;'''
curs = conn.cursor()
extras.execute_values(curs, insert_stmt, [tuple(i) for i in habitat.values])
conn.commit()

In [25]:
create_threats = """
CREATE TABLE threat (
taxonid INT REFERENCES assessment(taxonid),
code VARCHAR(6),
timing VARCHAR(24),
scope VARCHAR(17),
severity VARCHAR(32),
score VARCHAR(1),
invasive TEXT,
PRIMARY KEY (taxonid, code)
)
"""
curs = conn.cursor()
curs.execute(create_threats)
conn.commit()
curs.close()

In [26]:
insert_stmt = f'INSERT INTO threat ({",".join(list(threat))}) VALUES %s'
curs = conn.cursor()
extras.execute_values(curs, insert_stmt, [tuple(i) for i in threat.values])
conn.commit()

In [27]:
create_common_names = '''
CREATE TABLE common_name (
scientific VARCHAR(60) REFERENCES assessment(scientific_name),
common VARCHAR(42),
is_primary BOOLEAN,
language VARCHAR(3),
PRIMARY KEY (scientific, common)
)
'''
curs = conn.cursor()
curs.execute(create_common_names)
conn.commit()
curs.close()

In [12]:
create_historical = '''
CREATE TABLE history (
id SERIAL PRIMARY KEY,
taxonid int REFERENCES assessment(taxonid),
y2000 VARCHAR(2),
y2001 VARCHAR(2),
y2002 VARCHAR(2),
y2003 VARCHAR(2),
y2004 VARCHAR(2),
y2005 VARCHAR(2),
y2006 VARCHAR(2),
y2007 VARCHAR(2),
y2008 VARCHAR(2),
y2009 VARCHAR(2),
y2010 VARCHAR(2),
y2011 VARCHAR(2),
y2012 VARCHAR(2),
y2013 VARCHAR(2),
y2014 VARCHAR(2),
y2015 VARCHAR(2),
y2016 VARCHAR(2),
y2017 VARCHAR(2),
y2018 VARCHAR(2),
y2019 VARCHAR(2),
y2020 VARCHAR(2)
)
'''
curs = conn.cursor()
curs.execute(create_historical)
conn.commit()
curs.close()

In [13]:
insert_stmt = f'INSERT INTO history ({",".join([str(i) for i in list(historical)])}) VALUES %s'
curs = conn.cursor()
extras.execute_values(curs, insert_stmt, [tuple(i) for i in historical.values])
conn.commit()